# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


## 1. Download data from the Wikipedia page
We'll use the beautifulsoup package to download the table

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Get the text at the wikipedia website
wiki_page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

wiki_page_text = requests.get(wiki_page_url).text

In [3]:
# Use BeautifulSoup to isolate the table and save it as a list of strings
soup = BeautifulSoup(wiki_page_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})

each_row = table.text.split('\n')

In [4]:
# Iterate through each row to cfeate a list where each element is one "row" in the wikipedia table
index = 0
output = [[]]
for row in each_row:
    if row != "":
        output[index].append(row)
    if len(output[index]) == 3:
        index += 1
        output.append([])

In [5]:
# Convert the list to the dataframe
df = pd.DataFrame(output)

header = df.iloc[0]
df = df[1:]
df.columns = header

In [10]:
# Apply the transformations noted in the prompt. IE, removing "Not assbouroughs 
df = df[df['Borough'] !='Not assigned']

df.loc[(df['Neighbourhood'] == 'Not assigned'), 'Neighbourhood'] = df['Borough']

In [14]:
print('The shape of this dataframe is:\n{}'.format(df.shape))

The shape of this dataframe is:
(104, 3)
